In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
import os

In [2]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

In [3]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

# app.config['SQLALCHEMY_DATABASE_URI'] = DB_URL
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # silence the deprecation warning

# db = SQLAlchemy(app)

In [4]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [5]:
# TODO Fix SQL input/output; temp fix:
df = pd.read_pickle('data/kickstarter_data.pkl')

In [6]:
print(df.shape)
df.head()

(19291, 16)


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
ID,,,,,,,,,,,,,,,,
1031685482,You Sparkle Inside: A Kids' Book + Commissions...,Children's Books,Publishing,USD,2018-01-02,2000.0,2017-12-06,6083.00,successful,133,US,1054.0,6083.00,2000.0,27 days,304.150000
1176736360,THE NEW BILLY WYLDER RECORD,Indie Rock,Music,USD,2018-01-02,10000.0,2017-11-30,11169.56,successful,177,US,650.0,11169.56,10000.0,33 days,111.695600
1894703097,Driftwood Holly ***The Venice Project***,Indie Rock,Music,EUR,2018-01-02,25000.0,2017-11-28,25417.20,successful,168,DE,0.0,30615.02,30112.5,35 days,101.668809
311863059,On a Limb's debut album,Jazz,Music,USD,2018-01-02,1000.0,2017-12-09,1743.00,successful,39,US,0.0,1743.00,1000.0,24 days,174.300000
2076121120,Briggs Barbecue Sauces,Small Batch,Food,USD,2018-01-02,200000.0,2017-11-03,1.00,failed,1,US,0.0,1.00,200000.0,60 days,0.000500


## KNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

X = pd.get_dummies(df[['main_category','country','currency','usd_goal_real']])
# X['launched'] = df['launched'].apply(lambda x: int(x))
y = pd.get_dummies(df['state'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [17]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    print('KNN:{} Accuracy:{}'.format(i,metrics.accuracy_score(y_test, y_pred)))

KNN:1 Accuracy:0.5528054943630945
KNN:2 Accuracy:0.3191654788130102
KNN:3 Accuracy:0.5709472592976546
KNN:4 Accuracy:0.43553194246468835
KNN:5 Accuracy:0.5839056628223402
KNN:6 Accuracy:0.4775171698846702
KNN:7 Accuracy:0.5933652973953609
KNN:8 Accuracy:0.49151224569133084
KNN:9 Accuracy:0.5902552805494363
KNN:10 Accuracy:0.5020085525463263


## Logistic Regression

## SVM

## Decision Trees

## Random Forests